# Backend Server Deployment Guide

## Step 1: Check the available Node.js module streams

The backend service is written in Node.js. The project context says it works with Node.js 20 or newer, so review the available module streams first and confirm the required version if your environment has stricter standards.

```bash
dnf module list nodejs
```


## Step 2: Disable the default Node.js module

Some RHEL-based systems expose an older Node.js stream by default. Disable that stream so you can explicitly enable the required version.

```bash
dnf module disable nodejs -y
```


## Step 3: Enable the Node.js 20 module

Enable the Node.js 20 stream so the package manager installs the correct runtime for this application.

```bash
dnf module enable nodejs:20 -y
```


## Step 4: Install Node.js

Install the Node.js runtime and package manager. The application uses `npm` to install its dependencies later in the process.

```bash
dnf install nodejs -y
```


## Step 5: Create the application service user

Create a dedicated user named `expense` to run the backend service. This keeps the application separate from interactive user accounts and follows the original project convention.

```bash
useradd expense
```


## Step 6: Create the application directory

The backend code is deployed under `/app`. Create that directory before downloading and extracting the application files.

```bash
mkdir /app
```


## Step 7: Download the backend application archive

Fetch the packaged backend build and save it under `/tmp`. This keeps the downloaded archive separate from the application directory.

```bash
curl -o /tmp/backend.zip https://expense-builds.s3.us-east-1.amazonaws.com/expense-backend-v2.zip
```


## Step 8: Extract the application files into `/app`

Move into the application directory and unzip the backend package there so the service can run from `/app`.

```bash
cd /app
unzip /tmp/backend.zip
```


## Step 9: Install the application dependencies

The backend defines its library dependencies in the Node.js project files. Run `npm install` inside `/app` to download everything the service needs.

```bash
cd /app
npm install
```


## Step 10: Create the systemd service definition

Define a `systemd` unit so the backend can be managed with `systemctl`. Save the following content in `/etc/systemd/system/backend.service`.

```ini
[Unit]
Description=Backend Service

[Service]
User=expense
Environment=DB_HOST="<MYSQL-SERVER-IPADDRESS>"
ExecStart=/bin/node /app/index.js
SyslogIdentifier=backend

[Install]
WantedBy=multi-user.target
```

If you prefer to open the file manually:

```bash
vim /etc/systemd/system/backend.service
```


## Step 11: Reload systemd

After creating or changing a unit file, reload the `systemd` daemon so it picks up the new backend service definition.

```bash
systemctl daemon-reload
```


## Step 12: Start the backend service

Start the backend immediately so you can verify that the service launches successfully.

```bash
systemctl start backend
```


## Step 13: Enable the backend service at boot

Enable the service so the backend starts automatically whenever the server reboots.

```bash
systemctl enable backend
```


## Step 14: Install the MySQL client

The backend schema must be loaded into MySQL. Install the MySQL client first so the server can connect to the database and run the schema file.

```bash
dnf install mysql -y
```


## Step 15: Load the backend schema into MySQL

Import the schema file from the application directory into the MySQL server. This creates the database objects the backend expects.

```bash
mysql -h <MYSQL-SERVER-IPADDRESS> -uroot -pExpenseApp@1 < /app/schema/backend.sql
```


## Step 16: Restart the backend service

Restart the service after loading the schema so the application starts cleanly with the required database objects in place.

```bash
systemctl restart backend
```
